In [2]:
import pandas as pd
ponto = pd.read_csv('PontoColaboradores.txt', encoding='latin-1')


Filtrando dados apenas para a data de hoje (23/10/2025)

In [3]:
from datetime import date, timedelta

hoje = date.today()
ontem = hoje - timedelta(days=1)
print(f"Filtrando dados para: {ontem}")

Filtrando dados para: 2025-10-25


## Análise do arquivo de ponto



In [4]:

print("Primeiras 5 linhas do arquivo bruto:")
with open('PontoColaboradores.txt', 'r', encoding='latin-1') as f:
    for i in range(5):
        linha = f.readline()
        print(f"Linha {i + 1} (len={len(linha)}): {repr(linha[:200])}")

Primeiras 5 linhas do arquivo bruto:
Linha 1 (len=237): '00000000012   19018223972370700014182WALDEMIR IVAL LOTO                                                                                                                                    0001400375006'
Linha 2 (len=50): '000000001401012001000005022019111200000000000ae6f\n'
Linha 3 (len=25): '000000002606022019113806\n'
Linha 4 (len=25): '000000003606022019113902\n'
Linha 5 (len=25): '000000004606022019115702\n'


In [5]:
# Tenta identificar o padrão de data no arquivo
import re

print("\nBuscando padrões de data nas primeiras linhas:")
for idx in range(min(8, len(ponto))):
    linha = str(ponto.iloc[idx, 0])
    # Procura por datas no formato DDMMAAAA ou AAAAMMDD
    datas = re.findall(r'\d{8}', linha)
    if datas:
        print(f"Linha {idx}: Possíveis datas encontradas: {datas}")


Buscando padrões de data nas primeiras linhas:
Linha 0: Possíveis datas encontradas: ['00000000', '14010120', '01000005', '02201911', '12000000']
Linha 1: Possíveis datas encontradas: ['00000000', '26060220', '19113806']
Linha 2: Possíveis datas encontradas: ['00000000', '36060220', '19113902']
Linha 3: Possíveis datas encontradas: ['00000000', '46060220', '19115702']
Linha 4: Possíveis datas encontradas: ['00000000', '56150320', '19192502']
Linha 5: Possíveis datas encontradas: ['00000000', '62150320', '00000000', '36841897', '90037070']
Linha 6: Possíveis datas encontradas: ['00000000', '74150320', '19193515', '03201917', '29000000']
Linha 7: Possíveis datas encontradas: ['00000000', '85150320', '01273710', '00000000']


## Diagnóstico: Formato do arquivo de ponto

O arquivo parece ser de largura fixa e as datas extraídas não estão corretas. Vamos analisar a estrutura real do arquivo para identificar onde estão as datas.

## Análise da estrutura da linha

A linha fornecida mostra que a data está em duas partes:
- Posição 11-18: `15032019` (data DDMMAAAA = 15/03/2019)
- Posição 19-26: `19351503` (hora HHMM + repetição da data parcial)
- Continuação: `2019172900000000001c38a`

Vamos extrair corretamente os dados das linhas usando posições fixas.

In [6]:
# Analisa a linha fornecida pelo usuário
linha_exemplo = "000000007415032019193515032019172900000000001c38a"

print("Análise da linha fornecida:")
print(f"Linha completa: {linha_exemplo}")
print(f"\nSegmentação:")
print(f"Pos 0-9   (10 chars): '{linha_exemplo[0:10]}'   - ID/Sequência")
print(f"Pos 10-17 (8 chars):  '{linha_exemplo[10:18]}'  - Data (DDMMAAAA)")
print(f"Pos 18-21 (4 chars):  '{linha_exemplo[18:22]}'  - Hora (HHMM)")
print(f"Pos 22-29 (8 chars):  '{linha_exemplo[22:30]}'  - Data repetida")
print(f"Pos 30-33 (4 chars):  '{linha_exemplo[30:34]}'  - Hora repetida?")
print(f"Pos 34+:              '{linha_exemplo[34:]}'    - Dados adicionais")

Análise da linha fornecida:
Linha completa: 000000007415032019193515032019172900000000001c38a

Segmentação:
Pos 0-9   (10 chars): '0000000074'   - ID/Sequência
Pos 10-17 (8 chars):  '15032019'  - Data (DDMMAAAA)
Pos 18-21 (4 chars):  '1935'  - Hora (HHMM)
Pos 22-29 (8 chars):  '15032019'  - Data repetida
Pos 30-33 (4 chars):  '1729'  - Hora repetida?
Pos 34+:              '00000000001c38a'    - Dados adicionais


In [7]:
# Testa a extração da data e hora
from datetime import datetime

sequencia = linha_exemplo[0:10]
data_str = linha_exemplo[10:18]
hora_str = linha_exemplo[18:22]

print(f"\nExtração:")
print(f"Sequência: {sequencia}")
print(f"Data string: {data_str}")
print(f"Hora string: {hora_str}")

try:
    data = datetime.strptime(data_str, '%d%m%Y')
    hora = datetime.strptime(hora_str, '%H%M').time()
    data_hora = datetime.combine(data.date(), hora)
    print(f"\nData/hora extraída: {data_hora}")
except Exception as e:
    print(f"Erro ao extrair: {e}")


Extração:
Sequência: 0000000074
Data string: 15032019
Hora string: 1935

Data/hora extraída: 2019-03-15 19:35:00


In [8]:

registros = []

for idx in range(len(ponto)):
    linha = str(ponto.iloc[idx, 0])

    if len(linha) >= 22:
        try:
            sequencia = linha[0:10].strip()
            data_str = linha[10:18]
            hora_str = linha[18:22]

            data = datetime.strptime(data_str, '%d%m%Y')
            hora = datetime.strptime(hora_str, '%H%M').time()
            data_hora = datetime.combine(data.date(), hora)

            registros.append({
                'sequencia': sequencia,
                'data': data.date(),
                'hora': hora,
                'data_hora': data_hora,
                'linha_completa': linha
            })
        except:
            pass

df_ponto = pd.DataFrame(registros)
print(f"Total de registros extraídos: {len(df_ponto)}")
print(df_ponto.head(20))

# Salva a saída em um arquivo TXT
nome_arquivo_saida = 'registros_ponto_extraidos.txt'

try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as f:
        # Escreve cabeçalho
        f.write("=" * 80 + "\n")
        f.write("RELATÓRIO DE REGISTROS DE PONTO EXTRAÍDOS\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Total de registros extraídos: {len(df_ponto)}\n")
        f.write(f"Data de geração: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write("=" * 80 + "\n\n")

        # Escreve os dados do DataFrame formatados
        if len(df_ponto) > 0:
            f.write("REGISTROS:\n")
            f.write("-" * 80 + "\n")

            for idx, row in df_ponto.iterrows():
                f.write(f"Registro {idx + 1}:\n")
                f.write(f"  Sequência:  {row['sequencia']}\n")
                f.write(f"  Data:       {row['data'].strftime('%d/%m/%Y')}\n")
                f.write(f"  Hora:       {row['hora'].strftime('%H:%M')}\n")
                f.write(f"  Data/Hora:  {row['data_hora'].strftime('%d/%m/%Y %H:%M')}\n")
                f.write(f"  Linha:      {row['linha_completa']}\n")
                f.write("-" * 80 + "\n")

            # Adiciona resumo estatístico
            f.write("\n" + "=" * 80 + "\n")
            f.write("RESUMO ESTATÍSTICO\n")
            f.write("=" * 80 + "\n")
            f.write(f"Data mais antiga: {df_ponto['data'].min().strftime('%d/%m/%Y')}\n")
            f.write(f"Data mais recente: {df_ponto['data'].max().strftime('%d/%m/%Y')}\n")
            f.write(f"Total de dias únicos: {df_ponto['data'].nunique()}\n")
            f.write(f"Total de registros: {len(df_ponto)}\n")
        else:
            f.write("Nenhum registro foi extraído.\n")

    print(f"\n✓ Arquivo salvo com sucesso: {nome_arquivo_saida}")

except Exception as e:
    print(f"\n✗ Erro ao salvar arquivo: {e}")

df_ponto.head(20)

Total de registros extraídos: 212330
     sequencia        data      hora           data_hora  \
0   0000000014  2001-01-01  00:00:00 2001-01-01 00:00:00   
1   0000000026  2019-02-06  11:38:00 2019-02-06 11:38:00   
2   0000000036  2019-02-06  11:39:00 2019-02-06 11:39:00   
3   0000000046  2019-02-06  11:57:00 2019-02-06 11:57:00   
4   0000000056  2019-03-15  19:25:00 2019-03-15 19:25:00   
5   0000000062  2019-03-15  19:35:00 2019-03-15 19:35:00   
6   0000000074  2019-03-15  19:35:00 2019-03-15 19:35:00   
7   0000000085  2019-03-15  17:29:00 2019-03-15 17:29:00   
8   0000000095  2019-03-15  17:29:00 2019-03-15 17:29:00   
9   0000000105  2019-03-15  17:29:00 2019-03-15 17:29:00   
10  0000000115  2019-03-15  17:29:00 2019-03-15 17:29:00   
11  0000000125  2019-03-15  17:29:00 2019-03-15 17:29:00   
12  0000000135  2019-03-15  17:29:00 2019-03-15 17:29:00   
13  0000000145  2019-03-15  17:29:00 2019-03-15 17:29:00   
14  0000000155  2019-03-15  17:29:00 2019-03-15 17:29:00   
15 

,sequencia,data,hora,data_hora,linha_completa
0,0000000014,2001-01-01,00:00:00,2001-01-01 00:00:00,000000001401012001000005022019111200000000000ae6f
1,0000000026,2019-02-06,11:38:00,2019-02-06 11:38:00,000000002606022019113806
2,0000000036,2019-02-06,11:39:00,2019-02-06 11:39:00,000000003606022019113902
3,0000000046,2019-02-06,11:57:00,2019-02-06 11:57:00,000000004606022019115702
4,0000000056,2019-03-15,19:25:00,2019-03-15 19:25:00,000000005615032019192502
5,0000000062,2019-03-15,19:35:00,2019-03-15 19:35:00,0000000062150320191935 000000000012 368418...
6,0000000074,2019-03-15,19:35:00,2019-03-15 19:35:00,000000007415032019193515032019172900000000001c38a
7,0000000085,2019-03-15,17:29:00,2019-03-15 17:29:00,0000000085150320191729I012737109657ADAILSON WE...
8,0000000095,2019-03-15,17:29:00,2019-03-15 17:29:00,0000000095150320191729I016030497678ADEILSON SO...
9,0000000105,2019-03-15,17:29:00,2019-03-15 17:29:00,0000000105150320191729I020370904995ADRIANO BOG...


In [9]:
# Filtra registros para hoje
hoje = date.today()
ontem = hoje - timedelta(days=1)
df_hoje = df_ponto[df_ponto['data'] == ontem]

print(f"\nRegistros de hoje ({hoje}):")
print(f"Total: {len(df_hoje)}")
#df_hoje
nome_arquivo_saida = 'registros_ponto_hoje.txt'

# Salva registros de hoje em TXT
try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write(f"REGISTROS DE HOJE - {hoje.strftime('%d/%m/%Y')}\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Total de registros hoje: {len(df_hoje)}\n")
        f.write(f"Data de geração: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write("=" * 80 + "\n\n")

        if len(df_hoje) > 0:
            f.write("REGISTROS:\n")
            f.write("-" * 80 + "\n")
            for idx, row in df_hoje.iterrows():
                f.write(f"Registro {idx + 1}:\n")
                f.write(f"  Sequência:  {row['sequencia']}\n")
                f.write(f"  Data:       {row['data'].strftime('%d/%m/%Y')}\n")
                f.write(f"  Hora:       {row['hora'].strftime('%H:%M')}\n")
                f.write(f"  Data/Hora:  {row['data_hora'].strftime('%d/%m/%Y %H:%M')}\n")
                f.write(f"  Linha:      {row['linha_completa']}\n")
                f.write("-" * 80 + "\n")
        else:
            f.write("Nenhum registro encontrado para hoje.\n")

    print(f"\n✓ Arquivo de registros de hoje salvo com sucesso: {nome_arquivo_saida}")
except Exception as e:
    print(f"\n✗ Erro ao salvar arquivo de hoje: {e}")


Registros de hoje (2025-10-26):
Total: 99

✓ Arquivo de registros de hoje salvo com sucesso: registros_ponto_hoje.txt
